1.0. Libraries load.

In [2]:
# libraries load
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
import string
import re

2.0. Amazon Review dataset load.

In [4]:
from pyspark.sql.types import *

# Data frame schema definition
reviews_df_schema = StructType(
[StructField('marketplace', StringType()),
 StructField('customer_id', IntegerType()),
 StructField('review_id', StringType()),
 StructField('product_id', StringType()),
 StructField('product_parent', IntegerType()),
 StructField('product_title', StringType()),
 StructField('product_category', StringType()),
 StructField('star_rating', IntegerType()),
 StructField('helpful_votes', IntegerType()),
 StructField('total_votes', IntegerType()),
 StructField('vine', StringType()),
 StructField('verified_purchase', StringType()),
 StructField('review_headline', StringType()),
 StructField('review_body', StringType()),
 StructField('review_date', DateType())]
)

# Review dataset load
reviews_df = sqlContext.read.format('com.databricks.spark.csv')                                                           \
              .options(inferSchema=False, delimiter = '\t', nullValue='NA', header=True, encoding='UTF-8')                \
              .schema(reviews_df_schema)                                                                                  \
              .load( "/FileStore/tables/amazon_reviews_multilingual_US_v1_00_tsv-2db74.gz" )                              \
              .withColumn('product_title', lower(col('product_title')))                                                   \
              .withColumn('review_headline', lower(col('review_headline')))                                               \
              .withColumn('review_body', trim(col('review_body')))                                                        \
              .dropna()

# Save data frame in cache
reviews_df.cache()

# Exoplore review datafame
reviews_df.show(4)
#display(reviews_df.take(4))

print("Number of marketplace: " + str(reviews_df.select('marketplace').distinct().count()))
print("Number of product_category: " + str(reviews_df.select('product_category').distinct().count()))
print("Number of reviews: " + str(reviews_df.count()))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
marketplace|customer_id| review_id|product_id|product_parent| product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
 US| 53096384| R63J84G1LOX6R|1563890119| 763187671|the sandman vol. ...| Books| 4| 0| 1| N| N|ignore the review...|this is the first...| 1995-08-13|
 US| 53096399|R1BALOA11Z06MT|1559947608| 381720534|the 22 immutable ...| Books| 4| 0| 0| N| N| awesome|I've always been ...| 1995-08-17|
 US| 53096332|R1LLAY5W5PZUS4|0671701800| 860650224| contact| Books| 5| 1| 2| N| N|read the book. it...|This is a book ab...| 1995-08-30|
 US| 53096335|R3R9VTJ82FXECQ|0425132153| 624269601| good omens| Books| 5| 0| 0| N| N|funniest book eve...|This is quite pos...| 1995-09-11|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
only showing top 4 rows

Number of marketplace: 1
Number of product_category: 38
Number of reviews: 6930459

3.0. Review normalization.
- Punctuation, line break, numbers and special characters removed.

In [6]:
# Función eliminar puntuación
def remove_punct(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)  
    return nopunct

punct_remover = udf(lambda x: remove_punct(x))

# Ejecución funcción punct_remover sobre campo review
reviews_df_cleaned = reviews_df.select('marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', \
                                       punct_remover('product_title'), 'product_category', 'star_rating', 'helpful_votes', 'total_votes', \
                                       'vine', 'verified_purchase', punct_remover('review_headline'), punct_remover('review_body'), 'review_date')

# Renombrado y eliminación columnas
reviews_df_cleaned = reviews_df_cleaned.withColumnRenamed('<lambda>(product_title)', 'product_title').withColumn('product_title', trim(col('product_title')))
reviews_df_cleaned = reviews_df_cleaned.withColumnRenamed('<lambda>(review_headline)', 'review_headline').withColumn('review_headline', trim(col('review_headline')))
reviews_df_cleaned = reviews_df_cleaned.withColumnRenamed('<lambda>(review_body)', 'review_body').withColumn('review_body', trim(col('review_body')))

reviews_df_cleaned.show(4, truncate=100)


+-----------+-----------+--------------+----------+--------------+------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+------------------------------------------------+----------------------------------------------------------------------------------------------------+-----------+
marketplace|customer_id| review_id|product_id|product_parent| product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body|review_date|
+-----------+-----------+--------------+----------+--------------+------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+------------------------------------------------+----------------------------------------------------------------------------------------------------+-----------+
 US| 53096384| R63J84G1LOX6R|1563890119| 763187671|the sandman vol preludes and nocturnes| Books| 4| 0| 1| N| N| ignore the review below|this is the first issues of the series it is the starting point of all this it also contain...| 1995-08-13|
 US| 53096399|R1BALOA11Z06MT|1559947608| 381720534| the immutable laws of marketing| Books| 4| 0| 0| N| N| awesome|I ve always been partial to immutable laws The tape is entertaining good car listening The law...| 1995-08-17|
 US| 53096332|R1LLAY5W5PZUS4|0671701800| 860650224| contact| Books| 5| 1| 2| N| N| read the book it s good|This is a book about first contact with aliens written by a prominent member of SETI the Search...| 1995-08-30|
 US| 53096335|R3R9VTJ82FXECQ|0425132153| 624269601| good omens| Books| 5| 0| 0| N| N|funniest book ever written about the anti christ|This is quite possibly the funniest book I have ever read Terry Pratchett is an absolute geni...| 1995-09-11|
+-----------+-----------+--------------+----------+--------------+------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+------------------------------------------------+----------------------------------------------------------------------------------------------------+-----------+
only showing top 4 rows

3.0. Review Rating Conversion

In [8]:
# convert rating
def convert_rating(rating):
    if rating < 3: 
      return 'negative'
    elif rating == 3: 
        return 'neutral'
    else: 
      return 'positive'

# udf
rating_convert = udf(lambda x: convert_rating(x))

reviews_df_new = reviews_df_cleaned.select('marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', \
                                       'product_title', 'product_category', rating_convert('star_rating'), 'helpful_votes', 'total_votes', \
                                       'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date')

# Renombrado y eliminación columnas
reviews_df_new = reviews_df_new.withColumnRenamed('<lambda>(star_rating)', 'rating')

reviews_df_new.show(4)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+--------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
marketplace|customer_id| review_id|product_id|product_parent| product_title|product_category| rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+--------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
 US| 53096384| R63J84G1LOX6R|1563890119| 763187671|the sandman vol ...| Books|positive| 0| 1| N| N|ignore the review...|this is the first...| 1995-08-13|
 US| 53096399|R1BALOA11Z06MT|1559947608| 381720534|the immutable ...| Books|positive| 0| 0| N| N| awesome|I ve always been ...| 1995-08-17|
 US| 53096332|R1LLAY5W5PZUS4|0671701800| 860650224| contact| Books|positive| 1| 2| N| N|read the book it...|This is a book ab...| 1995-08-30|
 US| 53096335|R3R9VTJ82FXECQ|0425132153| 624269601| good omens| Books|positive| 0| 0| N| N|funniest book eve...|This is quite pos...| 1995-09-11|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+--------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
only showing top 4 rows

4.0. Selection of important columns

In [10]:
reviews_df_final= reviews_df_new.select('marketplace', 'product_id', 'product_title', 'product_category', 'rating', 'total_votes', \
                                       'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date')

reviews_df_final.show(4)

+-----------+----------+--------------------+----------------+--------+-----------+----+-----------------+--------------------+--------------------+-----------+
marketplace|product_id| product_title|product_category| rating|total_votes|vine|verified_purchase| review_headline| review_body|review_date|
+-----------+----------+--------------------+----------------+--------+-----------+----+-----------------+--------------------+--------------------+-----------+
 US|1563890119|the sandman vol ...| Books|positive| 1| N| N|ignore the review...|this is the first...| 1995-08-13|
 US|1559947608|the immutable ...| Books|positive| 0| N| N| awesome|I ve always been ...| 1995-08-17|
 US|0671701800| contact| Books|positive| 2| N| N|read the book it...|This is a book ab...| 1995-08-30|
 US|0425132153| good omens| Books|positive| 0| N| N|funniest book eve...|This is quite pos...| 1995-09-11|
+-----------+----------+--------------------+----------------+--------+-----------+----+-----------------+--------------------+--------------------+-----------+
only showing top 4 rows